# Cox–Ingersoll–Ross model

In mathematical finance, the Cox–Ingersoll–Ross (CIR) model describes the evolution of interest rates. It is a type of "one factor model" (short-rate model) as it describes interest rate movements as driven by only one source of market risk. The model can be used in the valuation of interest rate derivatives. It was introduced in 1985 by John C. Cox, Jonathan E. Ingersoll and Stephen A. Ross as an extension of the Vasicek model.

The CIR model specifies that the instantaneous interest rate $r_{t}$ follows the stochastic differential equation, also named the CIR Process:

$dr_t = a(b - r_t) dt + \sigma \sqrt{r_t} dW_t$

where 


$W_{t}$ is a Wiener process (modelling the random market risk factor) and a, b, and $\sigma$, are the parameters. 

The parameter 
- a corresponds to the speed of adjustment to the mean 
- b corresponds to long term mean $r_t$
- and $\sigma$, to volatility. 

**The drift factor, $a(b-r_{t})$, is exactly the same as in the Vasicek model**. It ensures mean reversion of the interest rate towards the long run value b, with speed of adjustment governed by the strictly positive parameter a.

The standard deviation factor, $\sigma {\sqrt  {r_{t}}}$, **avoids the possibility of negative interest rates for all positive values of a and b**. An interest rate of zero is also precluded/prevented if the condition

$2ab \ge \sigma^2$ is met

More generally, when the rate ($r_{t}$) is close to zero, the standard deviation ($\sigma {\sqrt  {r_{t}}}$) also becomes very small, which dampens the effect of the random shock on the rate. Consequently, when the rate gets close to zero, its evolution becomes dominated by the drift factor, which pushes the rate upwards (towards equilibrium).

This **process can be defined as a sum of squared Ornstein–Uhlenbeck process**. The CIR is an ergodic process, and possesses a stationary distribution. The **same process is used in the Heston model to model stochastic volatility**.

In [30]:
import QuantLib as ql
from collections import namedtuple
import math
import calibration_helper

In [31]:
today = ql.Date(15, ql.February, 2002);
settlement= ql.Date(19, ql.February, 2002);
ql.Settings.instance().evaluationDate = today;
term_structure = ql.YieldTermStructureHandle(ql.FlatForward(settlement,0.04875825,ql.Actual365Fixed()))
index = ql.Euribor1Y(term_structure)

In [32]:
CalibrationData = namedtuple("CalibrationData", 
                             "start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
        CalibrationData(2, 4, 0.1108),
        CalibrationData(3, 3, 0.1070),
        CalibrationData(4, 2, 0.1021),
        CalibrationData(5, 1, 0.1000 )]

In [33]:
# model = ql.CoxIngersollRoss(term_structure)
# TODO: fix
model = ql.ExtendedCoxIngersollRoss(term_structure)
engine = ql.TreeSwaptionEngine(model, 100)
swaptions = calibration_helper.create_swaption_helpers(data, index, term_structure, engine)

optimization_method = ql.LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = ql.EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)

print(model.params())
# print("a = %6.5f, sigma = %6.5f" % (a, sigma))

[ 0.0339478; 0.0962822; 0.0616989; 0.0110984 ]


In [34]:
calibration_helper.calibration_report(swaptions, data)

----------------------------------------------------------------------------------
    Model Price    Market Price     Implied Vol      Market Vol       Rel Error
----------------------------------------------------------------------------------
        0.00870         0.00949         0.10527         0.11480        -0.08290
        0.00968         0.01008         0.10644         0.11080        -0.03924
        0.00852         0.00872         0.10464         0.10700        -0.02197
        0.00646         0.00623         0.10602         0.10210         0.03824
        0.00356         0.00332         0.10702         0.10000         0.06989
----------------------------------------------------------------------------------
Cumulative Error :         0.12378
